In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install opencv-python-headless matplotlib scikit-image

In [4]:
# ===========================
# Setup: import library
# ===========================
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim
import os
from IPython.display import display, HTML

# ===========================
# 1) Path gambar di Google Drive
# ===========================
image_paths = [
    '/content/drive/MyDrive/ColabImages/photo1.png',
    '/content/drive/MyDrive/ColabImages/photo2.png',
    '/content/drive/MyDrive/ColabImages/photo3.png',
    '/content/drive/MyDrive/ColabImages/photo4.png',
    '/content/drive/MyDrive/ColabImages/photo5.png',
    '/content/drive/MyDrive/ColabImages/photo6.png',
]

# ===========================
# 2) Fungsi bantu
# ===========================
def to_gray(img_bgr):
    return cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

def plot_before_after_hist(orig, he, clahe, title=""):
    plt.figure(figsize=(12,6))
    plt.subplot(2,3,1); plt.imshow(orig, cmap='gray'); plt.title('Original'); plt.axis('off')
    plt.subplot(2,3,2); plt.imshow(he, cmap='gray'); plt.title('Equalized (HE)'); plt.axis('off')
    plt.subplot(2,3,3); plt.imshow(clahe, cmap='gray'); plt.title('CLAHE'); plt.axis('off')
    plt.subplot(2,3,4); plt.hist(orig.ravel(), bins=256, range=(0,255)); plt.title('Hist Original')
    plt.subplot(2,3,5); plt.hist(he.ravel(), bins=256, range=(0,255)); plt.title('Hist HE')
    plt.subplot(2,3,6); plt.hist(clahe.ravel(), bins=256, range=(0,255)); plt.title('Hist CLAHE')
    plt.suptitle(title)
    plt.tight_layout()
    plt.show()

def compute_ssim(a, b):
    return ssim(a, b)

# ===========================
# 3) Loop semua gambar
# ===========================
results = []

output_dir = 'results'
os.makedirs(output_dir, exist_ok=True)

for p in image_paths:
    if not os.path.exists(p):
        print(f"File not found: {p}")
        continue

    img_bgr = cv2.imread(p)
    gray = to_gray(img_bgr)

    he = cv2.equalizeHist(gray)

    clahe_op = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    clahe = clahe_op.apply(gray)

    ssim_he = compute_ssim(gray, he)
    ssim_clahe = compute_ssim(gray, clahe)

    base = os.path.splitext(os.path.basename(p))[0]
    cv2.imwrite(os.path.join(output_dir, base + '_he.jpg'), he)
    cv2.imwrite(os.path.join(output_dir, base + '_clahe.jpg'), clahe)

    plot_before_after_hist(gray, he, clahe, title=base)

    results.append({
        'file': p,
        'ssim_he': ssim_he,
        'ssim_clahe': ssim_clahe
    })

# ===========================
# 4) Ringkasan hasil
# ===========================
import pandas as pd
df = pd.DataFrame(results)
display(df)
df.to_csv(os.path.join(output_dir, 'summary_metrics.csv'), index=False)
print("Selesai. Hasil disimpan di folder:", output_dir)


Output hidden; open in https://colab.research.google.com to view.